In [ ]:
# IMPORT IMPORTANT LIBRARY


In [ ]:
def train_model_m0_name(hyperparameter, train_df_X, train_df_y):
    ''' Train and test a linear model for point forecasting. 
        
    Args:
        hyperparameter (df) : hyperparameter value of the model consisting of number of features
        train_df_X (df) : features matrix for training
        train_df_y (df) : target matrix for training

    
    Returns:
        model (model) : trained model with all features
    '''
    
    #UNPACK HYPERPARAMETER
    
    #TRAIN MODEL
    
  
    # PACK MODEL
    model = {"a": a, "b": b}
  

    return model

In [1]:
def produce_forecast_m0_name(model, train_df_X, test_df_X):
    """Create forecast at the train and test set using the trained model

    Args:
        model (dictionary): all parameters of the trained model
        train_df_X (df): predictors of train set
        test_df_X (df): predictors of test set

    Returns:
        train_df_y_hat (df) : forecast result at train set
        test_df_y_hat (df) : forecast result at test set
        
    """
    
    # UNPACK MODEL
    # fs_lr = model['feature_selector']
    # m06_lr = model['regression_model']
    
    # SELECT K BEST FEATURES
    # train_df_X = fs_lr.transform(train_df_X)
    # test_df_X = fs_lr.transform(test_df_X)
    
    # PRODUCE FORECAST
    # train_df_y_hat = m06_lr.predict(train_df_X)
    # test_df_y_hat = m06_lr.predict(test_df_X)
    
    return train_df_y_hat, test_df_y_hat